# Mounter

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Data Selector 4th level

In [0]:
import pandas as pd
import os
import json
import bz2

In [0]:
def merge(name_w,total):
  df=pd.read_csv(name_w)
  df['Hops']=total
  df.to_csv(name_w,index=False)

In [0]:
def extractor(file1,IP_list,writer,total,t_date,time):
  IP_dest='8.8.8.8'
  print(len(total))
  count=0
  prev="ip"
  temp_l=[]
  if not os.path.exists(file1):
    return total
  source_file=bz2.BZ2File(file1,'r')
  for i,line in enumerate(source_file):
    if i==6000000: 
      break
    if len(IP_list)==0:
      break
    data=json.loads(line)
    if 'from' not in data.keys() or 'dst_addr' not in data.keys():
      continue
    src_addr=data['from']
    dest_addr=data["dst_addr"]
    if dest_addr!='8.8.8.8' or src_addr not in IP_list:
      continue
    IP_list.remove(src_addr)
    print(len(IP_list))
    count+=1
    temp=data['result']
    for iter in temp:
      if "result" not in iter.keys():
        continue
      data_temp=iter['result']
      for ext in data_temp:
        if "from" in ext.keys():
          if prev==str(ext["from"]): continue
          temp_l.append(ext["from"])
          prev=str(ext["from"])
    if len(temp_l)==0:
      continue
    print(t_date)
    print(time)
    writer.write(str(i)+","+str(src_addr)+","+str(dest_addr)+","+str(t_date)+","+str(time)+"\n")
    total.append(temp_l)  
    temp_l=[]
  return total

In [0]:
def searcher(file_r):
  f=open(file_r,'r')
  total=[]
  Path="drive/My Drive/iitkgp_summer_project/"
  file_ext_1='Records_required.csv'
  dump_ext='total.json'
  writer=open(Path+file_ext_1,'w')
  writer.write("index,source,dest,date,time\n")
  writer.close()
  tot={}
  it=1
  Fldr='traceroute_2/'
  IP_req=json.loads(f.read())
  for date in IP_req.keys():
    t_date=date
    for time in IP_req[date].keys():
      writer=open(Path+file_ext_1,'a')
      IP_list=IP_req[date][time]["IP"]
      print(IP_list)
      if int(time) <10:
        time='0'+str(time)
      t_time='T'+str(time)+'00.bz2'
      t_date_ext=t_date[3:5]+'-'+t_date[0:2]
      t_date_2='2020-'+t_date_ext+t_time
      file_ext='traceroute-'+t_date_2
      file1=Path+Fldr+t_date+'/'+file_ext
      print(file1)
      total=extractor(file1,IP_list,writer,total,t_date,time)
      tot[str(it)]={}
      tot[str(it)]['Date']=t_date
      tot[str(it)]['Time']=time
      tot[str(it)]["IP"]=total
      it+=1
      with open(Path+dump_ext, 'w') as fp:
        json.dump(tot, fp)
      writer.close()
  merge(Path+file_ext_1,total)


In [0]:
searcher("drive/My Drive/iitkgp_summer_project/requirement_3rd.json")

['102.134.194.250', '109.105.133.166', '134.249.119.130', '176.163.68.130', '178.217.173.52', '178.236.131.68', '185.201.196.20', '188.32.235.65', '190.15.201.37', '195.135.215.2', '2.83.109.66', '24.123.25.203', '31.129.213.127', '46.140.240.40', '5.227.7.177', '5.51.202.112', '79.159.213.81', '79.169.124.118', '81.243.90.126', '81.34.44.194', '82.64.223.69', '86.154.254.213', '89.248.190.164', '90.154.71.149']
drive/My Drive/iitkgp_summer_project/traceroute_2/15-04/traceroute-2020-04-15T0800.bz2
0
23
15-04
08
22
15-04
08
21
15-04
08
20
15-04
08
19
15-04
08
18
15-04
08
17
15-04
08
16
15-04
08
15
15-04
08
14
15-04
08
13
15-04
08
12
15-04
08
11
15-04
08
10
15-04
08
9
15-04
08
8
15-04
08
7
15-04
08
6
15-04
08
5
15-04
08
4
15-04
08
3
15-04
08
2
15-04
08
1
15-04
08
0
15-04
08
['103.202.216.212', '108.202.150.53', '109.148.70.93', '154.126.208.20', '188.32.216.170', '195.135.215.2', '2.83.109.66', '212.26.130.38', '213.248.4.27', '217.18.40.3', '37.26.174.124', '5.227.7.177', '5.51.202.112'

# Grouping

In [0]:
import json
import os
import pandas as pd
import ast

In [0]:
def grouping(file1,file2):
  Path="drive/My Drive/iitkgp_summer_project/"
  file_ext="merged_file_3rd.csv"
  file_main=Path+file_ext
  df=pd.read_csv(file1)
  j=0
  #while j<len(df):
    #df.iloc[j]['Time']=int(df.iloc[j]['Time'][1:3])
    #j+=1
  df2=pd.read_csv(file2)
  df_temp=df2.drop([0,2])
  df_k=df_temp[['source','time','date','Hops']].values
  #np_k=df_k.values
  clmn=['Source','Time','date','Hops']
  df_2=pd.DataFrame(df_k,columns=clmn)
  df1=pd.concat([df,df_2])
  #df1=df1.sort_values(by=['Source,date,Time'])
  df1.to_csv(file_main,index=False)
  

In [0]:
def grouping_exec():
  Path="drive/My Drive/iitkgp_summer_project/"
  exec_1="final_3_gap.csv"
  exec_2="Records_required.csv"
  file1=Path+exec_1
  file2=Path+exec_2
  grouping(file1,file2)


In [0]:
grouping_exec()

In [0]:
def sort_list():
  Path="drive/My Drive/iitkgp_summer_project/"
  Exec="merged_file_3rd.csv"
  File=Path+Exec
  df=pd.read_csv(File)
  df1=df.sort_values(["Source","date","Time"])

  df1.to_csv(File,index=False)

In [0]:
sort_list()

# Sort and Eliminate

In [0]:
import pandas as pd
import json
import os

In [0]:
def eliminate_exec(file1):
  df=pd.read_csv(file1)
  Path="drive/My Drive/iitkgp_summer_project/"
  text=open(Path+"similar_removed_3rd.csv",'w')
  text.write('Source,Time,date,Hops\n')
  text.close()
  df2=pd.read_csv(Path+"similar_removed_3rd.csv")
  i=0
  new_l=[]
  cl=['Source','Time','date','Hops']
  current=""
  while i<1056:
    if df.iloc[i]['Source']!=current:
        current=df.iloc[i]['Source']
        print("S")
        df_temp=pd.DataFrame(new_l,columns=cl)
        flag=0
        df_temp=df_temp.sort_values(by=['date','Time'])
        df_temp=eliminate(df_temp)
        if len(df_temp!=0):
          df2=pd.concat([df2,df_temp],axis=0)
        new_l=[]
       #i+=1
    else:
      while df.iloc[i]['Source']==current:
        print(i)
        
        temp_list=[df.iloc[i]['Source'],df.iloc[i]['Time'],df.iloc[i]['date'],df.iloc[i]['Hops']]
        
        new_l.append(temp_list)
        
        i+=1
        print(i)
        print(df.iloc[i]['Source'])
        if i==1056: break
  print(len(df2))
  df2.to_csv(Path+"similar_removed_3rd.csv",index=False) 

In [0]:
def eliminate(df_temp):
  print(df_temp)
  print(len(df_temp))
  l=len(df_temp)
  i=0
  j=1
  flag=0
  while j<len(df_temp):
    print("J    "+str(j))
    print("I    "+str(i))
    if df_temp.iloc[j]["Hops"]==df_temp.iloc[i]["Hops"]:
      if flag==0:
        print("A")
        df_temp=df_temp.drop([i])
        if j==l-1:
          print("B")
          df_temp=df_temp.drop([j])
      else:
        flag=0
        
        if j==l-1:
          print("C")
          df_temp=df_temp.drop([j])
    else:
      flag=1
    i+=1
    j+=1
  return df_temp


In [0]:
Path="drive/My Drive/iitkgp_summer_project/"
Exec="merged_file_3rd.csv"
eliminate_exec(Path+Exec)

Streaming output truncated to the last 5000 lines.
I    0
A
283
284
185.249.152.111
284
285
185.249.152.111
285
286
185.249.152.111
286
287
185.249.152.111
287
288
185.249.152.111
288
289
185.249.152.111
289
290
185.249.152.111
290
291
185.249.152.111
291
292
185.249.152.111
292
293
185.249.152.111
293
294
185.249.152.111
294
295
185.249.152.111
295
296
185.39.18.254
S
             Source  ...                                               Hops
0   185.249.152.111  ...  ['192.168.69.1', '185.249.152.66', '185.249.15...
1   185.249.152.111  ...  ['192.168.69.1', '185.249.152.66', '185.249.15...
2   185.249.152.111  ...  ['192.168.69.1', '185.249.152.66', '185.249.15...
3   185.249.152.111  ...  ['192.168.69.1', '185.249.152.66', '185.249.15...
4   185.249.152.111  ...  ['192.168.69.1', '185.249.152.66', '185.249.15...
5   185.249.152.111  ...  ['192.168.69.1', '185.249.152.66', '185.249.15...
6   185.249.152.111  ...  ['192.168.69.1', '185.249.152.66', '185.249.15...
7   185.249.152.111 

In [0]:
def eliminate_date_gap(file1):
  df=pd.read_csv(file1)
  Path="drive/My Drive/iitkgp_summer_project/"
  text=open(Path+"date_gap_removed_3rd.csv",'w')
  text.write('Source,Time,date,Hops,Time_gap\n')
  text.close()
  df2=pd.read_csv(Path+"date_gap_removed_3rd.csv")
  i=0
  new_l=[]
  cl=['Source','Time','date','Hops']
  current=""
  while i<873:
    if df.iloc[i]['Source']!=current:
        current=df.iloc[i]['Source']
        print("S")
        df_temp=pd.DataFrame(new_l,columns=cl)
        flag=0
        df_temp=df_temp.sort_values(by=['date','Time'])
        df_temp=eliminate_date(df_temp)
        if len(df_temp!=0):
          df2=pd.concat([df2,df_temp],axis=0)
        new_l=[]
       #i+=1
    else:
      while df.iloc[i]['Source']==current:
        print(i)
        
        temp_list=[df.iloc[i]['Source'],df.iloc[i]['Time'],df.iloc[i]['date'],df.iloc[i]['Hops']]
        
        new_l.append(temp_list)
        
        i+=1
        print(i)
        print(df.iloc[i]['Source'])
        if i==873: break
  print(len(df2))
  df2.to_csv(Path+"date_gap_removed_3rd.csv",index=False) 

In [0]:
def eliminate_date(df_temp):
  print(df_temp)
  print(len(df_temp))
  l=len(df_temp)
  time_g=[-1]
  i=0
  j=1
  flag=0
  while j<len(df_temp):
    print("J    "+str(j))
    print("I    "+str(i))
  
    day_diff=int(df_temp.iloc[j]["date"][0:2])-int(df_temp.iloc[i]["date"][0:2])
    if day_diff==0:
      time_gap=int(df_temp.iloc[j]['Time'])-int(df_temp.iloc[i]['Time'])
    elif day_diff==1:
      time_gap=24-int(df_temp.iloc[i]['Time'])+int(df_temp.iloc[j]['Time'])
    else:
      time_gap=day_diff*24+24-int(df_temp.iloc[i]['Time'])+int(df_temp.iloc[j]['Time'])
    time_g.append(time_gap)
    i+=1
    j+=1
  df_temp['Time_gap']=time_g
  return df_temp


In [0]:
Path="drive/My Drive/iitkgp_summer_project/"
Exec="similar_removed_3rd.csv"
eliminate_date_gap(Path+Exec)

Streaming output truncated to the last 5000 lines.
119
120
151.20.3.242
S
            Source  ...                                               Hops
0  150.241.254.254  ...  ['150.241.254.1', '150.241.253.149', '185.12.1...
1  150.241.254.254  ...  ['150.241.254.1', '150.241.253.149', '185.12.1...

[2 rows x 4 columns]
2
J    1
I    0
120
121
151.20.3.242
121
122
152.195.206.35
S
         Source  Time   date                                               Hops
0  151.20.3.242   8.0  13-04  ['192.168.1.1', '151.6.139.25', '151.6.250.120...
1  151.20.3.242   9.0  13-04  ['192.168.1.1', '151.6.250.120', '151.6.44.128...
2
J    1
I    0
122
123
152.195.206.35
123
124
154.126.208.20
S
           Source  ...                                               Hops
0  152.195.206.35  ...  ['152.195.206.33', '187.16.216.55', '74.125.24...
1  152.195.206.35  ...  ['152.195.206.33', '187.16.218.58', '108.170.2...

[2 rows x 4 columns]
2
J    1
I    0
124
125
154.126.208.20
125
126
154.126.208.20
126
127

In [0]:
def list_dates_lt3(file1):
  Path="drive/My Drive/iitkgp_summer_project/"
  ext="gap_removed_final_lt3.csv"
  File=Path+ext
  df=pd.read_csv(file1)
  print(len(df))
  temp=[]
  dict_k={}
  #df=df.sort_values(['Source','date','Time'])
  i=0
  j=1
  flag=0
  while j<len(df):
    key=df.iloc[j]['Time_gap']
    if key<3:
      if dict_k.get(str(i)) is None:
        dict_k[str(i)]=0
      if dict_k.get(str(j)) is None:
        dict_k[str(j)]=0
    i+=1
    j+=1
  k=0
  while k<len(df):
    if dict_k.get(str(k)) is None:
      temp.append(k)
    k+=1

  print(len(temp))
  df_k=df.drop(temp)
  print(len(df_k))
  df_k=df_k.drop(["Time_gap"],axis=1)
  df_k.to_csv(File,index=False)

In [0]:
Path="drive/My Drive/iitkgp_summer_project/"
Exec="date_gap_removed_3rd.csv"
list_dates_lt3(Path+Exec)

871
54
817
